# Variables categóricas

En esta clase vamos a ver como utilizar pandas y scikit learn para transformar variables categóricas en algo que los modelos de machine learning puedan entender.

Vamos a utilizar un dataset armado a mano y bastante simple para aprender a utilizar scikit learn y pandas.

Luego, tendrán que aplicar lo aprendido sobre el dataset de la clase pasada (ecommerce).

In [ ]:
#from google.colab import drive # La usamos para montar nuestra unidad de Google Drive
#drive.mount('/content/drive') # Montamos nuestra unidad de Google Drive

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = {'Temperature': ['Hot', 'Cold', 'Very Hot', 'Warm', 'Hot', 'Warm', 'Warm', 'Hot', 'Hot', 'Cold'],
        'Color': ['Red', 'Yellow','Blue', 'Blue', 'Red', 'Yellow', 'Red', 'Yellow', 'Yellow', 'Blue']}

df = pd.DataFrame(data)

In [3]:
df.Color.nunique()

3

## One hot encoding

En este simple caso, vemos que la variable Temperature puede ser considerada ordinal porque la temperatura va desde cold hasta very hot.

Por otro lado, en la variable color no vemos ningún orden, no podemos considerarla ordinal.

Vamos a aplicar one hot encoding en la variable color.

Esto se puede hacer con pandas o con el OneHotEncoder de scikit learn.

Comencemos con pandas.

Pandas nos brinda la funcion get_dummies():

In [4]:
pd.get_dummies(df.Color)

,Blue,Red,Yellow
0,False,True,False
1,False,False,True
2,True,False,False
3,True,False,False
4,False,True,False
5,False,False,True
6,False,True,False
7,False,False,True
8,False,False,True
9,True,False,False


¿ Cómo agregamos estas columnas a nuestro dataset ?

Podemos concatenar horizontalmente este dataset de variables dummies a el original:

La próxima clase veremos en más detalle los métodos concat, merge, etcétera.

In [5]:
dummies = pd.get_dummies(df.Color) # Obtenemos dummies
df_encoded = pd.concat([df, dummies], axis=1) # Concatenamos horizontalmente con axis=1

df_encoded

,Temperature,Color,Blue,Red,Yellow
0,Hot,Red,False,True,False
1,Cold,Yellow,False,False,True
2,Very Hot,Blue,True,False,False
3,Warm,Blue,True,False,False
4,Hot,Red,False,True,False
5,Warm,Yellow,False,False,True
6,Warm,Red,False,True,False
7,Hot,Yellow,False,False,True
8,Hot,Yellow,False,False,True
9,Cold,Blue,True,False,False


Ahora podemos eliminar la columna original

In [6]:
df_encoded = df_encoded.drop('Color', axis=1)

In [7]:
df_encoded

,Temperature,Blue,Red,Yellow
0,Hot,False,True,False
1,Cold,False,False,True
2,Very Hot,True,False,False
3,Warm,True,False,False
4,Hot,False,True,False
5,Warm,False,False,True
6,Warm,False,True,False
7,Hot,False,False,True
8,Hot,False,False,True
9,Cold,True,False,False


¿ Cómo hacemos lo mismo con scikit learn ?

Tenemos el OneHotEncoder en el módulo de preprocessing:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder



In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
# handle_unknow: si se encuentra con una categoria desconocida entonces completa todas las columnas con 0's
# sparse matrix ??? que nos dice? para que se usa?
# sparse bool, default=True Will return sparse matrix if set True else will return an array.

Averiguar en la documentación:
- ¿ Qué significa el `handle_unknown='ignore'` ?
- Que es "sparse" ?
- Que sucede si ponemos sparse = True?

Hacemos fit:

In [10]:
enc.fit(df.Color.values.reshape(-1,1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(handle_unknown='ignore', sparse=False, sparse_output=False)

In [11]:
df.Color.values.reshape(-1,1)

array([['Red'],
       ['Yellow'],
       ['Blue'],
       ['Blue'],
       ['Red'],
       ['Yellow'],
       ['Red'],
       ['Yellow'],
       ['Yellow'],
       ['Blue']], dtype=object)

- ¿ Qué pasa si sacamos el .reshape(-1, 1) ? -----> se rompe

- ¿ Qué otra forma se les ocurre para solucionar el error sin usar reshape ?

In [12]:
encoded_color = enc.transform(df.Color.values.reshape(-1, 1))
encoded_color

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]])

Ahora, como agregamos esto a nuestro dataframe?

El método get_feature_names nos da los nombres de las nuevas features creadas:

In [13]:
enc.get_feature_names_out(['Colores'])

array(['Colores_Blue', 'Colores_Red', 'Colores_Yellow'], dtype=object)

In [14]:
encoded_color_columns = enc.get_feature_names_out(['Colores'])

In [15]:
encoded_color_df = pd.DataFrame(data=encoded_color, columns= encoded_color_columns)
encoded_color_df

,Colores_Blue,Colores_Red,Colores_Yellow
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
5,0.0,0.0,1.0
6,0.0,1.0,0.0
7,0.0,0.0,1.0
8,0.0,0.0,1.0
9,1.0,0.0,0.0


Ahora, como hicimos antes, podemos concatenar y eliminar la columna original:

In [16]:
pd.concat([df, encoded_color_df], axis=1).drop('Color', axis=1)

,Temperature,Colores_Blue,Colores_Red,Colores_Yellow
0,Hot,0.0,1.0,0.0
1,Cold,0.0,0.0,1.0
2,Very Hot,1.0,0.0,0.0
3,Warm,1.0,0.0,0.0
4,Hot,0.0,1.0,0.0
5,Warm,0.0,0.0,1.0
6,Warm,0.0,1.0,0.0
7,Hot,0.0,0.0,1.0
8,Hot,0.0,0.0,1.0
9,Cold,1.0,0.0,0.0


Muchas veces, en lugar de crear todas las columnas, se utiliza el atributo `drop='first'`.

Esto crea todas las columnas menos la primera (en nuestro caso no se crearía color_blue), esto es porque si ninguna de las otras es 1, significa que blue es 1. Sirve para ahorrarnos una columna.

En el caso de variables binarias, podemos crear una única columna utilizando:

`drop='if_binary'`

## Label encoder

Se utiliza de una forma muy similar a el OneHotEncoder de scikit learn.

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() #creo el objeto

df['Temperature_label_encoded'] = le.fit_transform(df.Temperature)
df

,Temperature,Color,Temperature_label_encoded
0,Hot,Red,1
1,Cold,Yellow,0
2,Very Hot,Blue,2
3,Warm,Blue,3
4,Hot,Red,1
5,Warm,Yellow,3
6,Warm,Red,3
7,Hot,Yellow,1
8,Hot,Yellow,1
9,Cold,Blue,0


No se utiliza para datos ordinales ya que scikit learn le asigna un valor numérico pero sin tener en cuenta que nosotros queremos que cold sea menor que hot.

Cuando queremos especificar nosotros los valores numéricos para cada valor de la variable categórica, podemos utilizar la función .replace() de pandas.

Esta función recibe un diccionario en el que la key tiene que ser el valor que queremos transformar y el value el valor resultante que queremos.

Veamos un ejemplo:

In [ ]:
df.Temperature.unique()

array(['Hot', 'Cold', 'Very Hot', 'Warm'], dtype=object)

In [ ]:
mapping_dict = {
    'Cold': 1,
    'Warm': 2,
    'Hot': 3,
    'Very Hot': 4
}

temperature_ordinal = df.Temperature.replace(mapping_dict)
temperature_ordinal

0    3
1    1
2    4
3    2
4    3
5    2
6    2
7    3
8    3
9    1
Name: Temperature, dtype: int64

In [ ]:
df['Temperature_ordinal'] = temperature_ordinal
df

,Temperature,Color,Temperature_label_encoded,Temperature_ordinal
0,Hot,Red,1,3
1,Cold,Yellow,0,1
2,Very Hot,Blue,2,4
3,Warm,Blue,3,2
4,Hot,Red,1,3
5,Warm,Yellow,3,2
6,Warm,Red,3,2
7,Hot,Yellow,1,3
8,Hot,Yellow,1,3
9,Cold,Blue,0,1


In [ ]:
df.drop('Temperature_label_encoded', axis=1, inplace=True)

In [ ]:
df

,Temperature,Color,Temperature_ordinal
0,Hot,Red,3
1,Cold,Yellow,1
2,Very Hot,Blue,4
3,Warm,Blue,2
4,Hot,Red,3
5,Warm,Yellow,2
6,Warm,Red,2
7,Hot,Yellow,3
8,Hot,Yellow,3
9,Cold,Blue,1


# Discretización

Vamos a ver como hacerlo con sklearn. Para este caso vamos a utilizar otro dataset con una variable continua:

Creamos el dataset:

In [ ]:
variable_continua = np.arange(200)
df_cont = pd.DataFrame({'X': variable_continua})

In [ ]:
df_cont.head()

,X
0,0
1,1
2,2
3,3
4,4


Aplicamos KBinsDiscretizer.

Tenemos que pasarle la cantidad de bins, encode y strategy.

Averiguar que significan estos parametros:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy = 'uniform')

In [ ]:
# n_bins: cantidad de "cajas"(bines) en donde guardo mis datos
# encode: metodo usado para la transformación --- ‘onehot’ (nos devuelva sparse matrix) , ‘onehot-dense’ dense array , 'ordinal' te devuelve el  "bin identifier encoded"
# strategy:‘uniform’ todos los bines tinene el mismo ancho
#          :‘quantile’ todos los bines tienen la misma cant de datos
#          :'Values' in each bin have the same nearest center of a 1D k-means cluster.

En scikit learn siempre estuvimos aplicando el método fit y transform por separado. Scikit lern nos permite aplicar los dos en una linea con el método fit_transform:

In [ ]:
df_cont['discretized'] = est.fit_transform(df_cont[['X']]) # Acá en lugar de hacer reshape(-1, 1) utilizamos doble [[]]

In [ ]:
est.fit_transform(df_cont[['X']])

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [ ]:
df_cont

,X,discretized
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
195,195,4.0
196,196,4.0
197,197,4.0
198,198,4.0


In [ ]:
df_cont.discretized.value_counts()

discretized
0.0    40
1.0    40
2.0    40
3.0    40
4.0    40
Name: count, dtype: int64

# Ejercicio

Vamos a levantar el dataset de la clase pasada (esta vez sin nulos) y transformar las variables categóricas.

Tienen que utilizar su criterio para decidir cuando conviene ordinal, one hot, etc.

Recuerden que las columnas del dataset son:


- id: Id del usuario
- administrative: Número de veces que el usuario visito la sección "administrative"
- administrative_duration: Tiempo que el usuario paso en la sección administrative
- informational: Número de veces que el usuario visitó la sección "informational"
- informational_duration: Tiempo que el usuario paso en la sección informational
- productrelated: Número de veces que el usuario visitó la sección "products related"
- productrelated_duration: Tiempo que el usuario pasó en la sección
- bouncerates: Porcentaje de visitantes que entran a la página e inmediatamente la dejan sin interactuar con la misma. Esta metrica solo se tiene en cuenta si es la primer página que se visitó del sitio web.
- exitrates: De la cantidad total de visitas a las páginas del sitio web, el porcentage de usuarios que lo abandonaron en esta página. Esto es, el  porcentaje de usuarios que su última visita al sitio fué en esta página.
- pagevalues: Este es el valor promedio del sitio web, indica la contribución que este sitio web hizo al visitante que llega a la página o sección de compra final.
-  specialday: Es una fecha especial o no (1 o 0)
- operatingsystems: Sistema operativo
- browser: Nombre del navegador
- region: Region geográfica del usuario
- traffictype: Tipo de tráfico web
- visitortype: Nuevo o uno que retorno al sitio
- Weekend: 1 si es fin de semana y 0 en otro caso
- revenue: 1 si el usuario hizo una compra y 0 en otro caso

In [ ]:
df = pd.read_csv('online-shoppers-intention-sin-nulos.csv') #La salida de la clase pasada

In [ ]:
df.head()

Transformar las variables:

- Month
- Visitor type
- weekend

Con los métodos que aprendimos.

Discretizar:
- ExitRates
- BounceRates


Investigar:

- ¿Cómo puedo saber desde que valor hasta que valor van cada uno de los "bins" en KBinsDiscretizer? (buscar los atributos del discretizer en la documentación)
- ¿ Qué pasa si en lugar de usar encode="ordinal" uso encode=‘onehot’ o ‘onehot-dense’?
- ¿Cuál es la diferencia entre strategy=‘uniform’ y strategy=‘quantile’ ?